subset selection

In [1]:
#Some basic setups
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
#Replacement for ISLR2 library
import statsmodels.api as sm
# different approach with sklearn
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)


#We can try using get_dummies from panda library for binarization.

#Replacement for FNN library for KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

#Replacement for leaps library
from sklearn.feature_selection import SelectKBest, f_regression
from mlxtend.feature_selection import ExhaustiveFeatureSelector

#Replacement for caTools library
from sklearn.model_selection import train_test_split

#Replacement for glmnet library for CV
from sklearn.linear_model import ElasticNetCV

#Replacement for glmnet library for Ridge and Lasso
from sklearn.linear_model import ElasticNet

#Replacement for pls library
from sklearn.cross_decomposition import PLSRegression


In [7]:
df = pd.read_csv("/content/dataQTM.csv")
#M = 0, B = 1
y = df['diagnosis']
X = df.drop(['diagnosis', 'id'], axis=1).astype('float64')

# Define the feature set X.
Xcons = X
Xcons = sm.add_constant(Xcons)
print(Xcons)

     const  radius_0ean  texture_0ean  peri0eter_0ean  area_0ean  \
0      1.0        17.99         10.38          122.80     1001.0   
1      1.0        20.57         17.77          132.90     1326.0   
2      1.0        19.69         21.25          130.00     1203.0   
3      1.0        11.42         20.38           77.58      386.1   
4      1.0        20.29         14.34          135.10     1297.0   
..     ...          ...           ...             ...        ...   
564    1.0        21.56         22.39          142.00     1479.0   
565    1.0        20.13         28.25          131.20     1261.0   
566    1.0        16.60         28.08          108.30      858.1   
567    1.0        20.60         29.33          140.10     1265.0   
568    1.0         7.76         24.54           47.92      181.0   

     s0oothness_0ean  co0pactness_0ean  concavity_0ean  concave points_0ean  \
0            0.11840           0.27760         0.30010              0.14710   
1            0.08474     

In [8]:
#Explorative Analysis
df['diagnosis'].value_counts()

1    357
0    212
Name: diagnosis, dtype: int64

In [9]:
correlation_matrix = df.corr()
print(correlation_matrix)

                               id  diagnosis  radius_0ean  texture_0ean  \
id                       1.000000  -0.039769     0.074626      0.099770   
diagnosis               -0.039769   1.000000    -0.730029     -0.415185   
radius_0ean              0.074626  -0.730029     1.000000      0.323782   
texture_0ean             0.099770  -0.415185     0.323782      1.000000   
peri0eter_0ean           0.073159  -0.742636     0.997855      0.329533   
area_0ean                0.096893  -0.708984     0.987357      0.321086   
s0oothness_0ean         -0.012968  -0.358560     0.170581     -0.023389   
co0pactness_0ean         0.000096  -0.596534     0.506124      0.236702   
concavity_0ean           0.050080  -0.696360     0.676764      0.302418   
concave points_0ean      0.044158  -0.776614     0.822529      0.293464   
sy00etry_0ean           -0.022114  -0.330499     0.147741      0.071401   
fractal_di0ension_0ean  -0.052511   0.012838    -0.311631     -0.076437   
radius_se                

In [21]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate rsquared
    if 'const' not in feature_set:
        feature_set = feature_set + ('const',) # mannually add intercept
    model = sm.OLS(y,Xcons[list(feature_set)])
    regr = model.fit()
    rsquared = regr.rsquared
    return {"model":regr, "rsquared":rsquared}

In [22]:
import time
import itertools

In [23]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]

    tic = time.time()

    results = []

    for p in remaining_predictors:
        results.append(processSubset(tuple(predictors+[p])))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest rsquared
    best_model = models.loc[models['rsquared'].argmax()]

    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")

    # Return the best model, along with some other useful information about the model
    return best_model

In [25]:
models_fwd = pd.DataFrame(columns=["rsquared", "model"])

tic = time.time()
predictors = []

for i in range(1,31):
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  30 models on 1 predictors in 0.10165095329284668 seconds.
Processed  29 models on 3 predictors in 0.03768324851989746 seconds.
Processed  28 models on 4 predictors in 0.03806138038635254 seconds.
Processed  27 models on 5 predictors in 0.03529834747314453 seconds.
Processed  26 models on 6 predictors in 0.037348031997680664 seconds.
Processed  25 models on 7 predictors in 0.049327850341796875 seconds.
Processed  24 models on 8 predictors in 0.03560185432434082 seconds.
Processed  23 models on 9 predictors in 0.03415560722351074 seconds.
Processed  22 models on 10 predictors in 0.03455924987792969 seconds.
Processed  21 models on 11 predictors in 0.030813217163085938 seconds.
Processed  20 models on 12 predictors in 0.034436702728271484 seconds.
Processed  19 models on 13 predictors in 0.039566993713378906 seconds.
Processed  18 models on 14 predictors in 0.03690147399902344 seconds.
Processed  17 models on 15 predictors in 0.02881312370300293 seconds.
Processed  16 models on

In [26]:
print(models_fwd.loc[21, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:              diagnosis   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     89.01
Date:                Wed, 13 Dec 2023   Prob (F-statistic):          4.71e-161
Time:                        04:08:42   Log-Likelihood:                 28.752
No. Observations:                 569   AIC:                            -13.50
Df Residuals:                     547   BIC:                             82.06
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
concave points_worst      

In [27]:
print(models_fwd.loc[10, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:              diagnosis   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     176.2
Date:                Wed, 13 Dec 2023   Prob (F-statistic):          1.85e-165
Time:                        04:08:46   Log-Likelihood:                 11.562
No. Observations:                 569   AIC:                            -1.124
Df Residuals:                     558   BIC:                             46.66
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
concave points_worst      

In [28]:
print(models_fwd.loc[13, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:              diagnosis   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     141.8
Date:                Wed, 13 Dec 2023   Prob (F-statistic):          9.50e-167
Time:                        04:08:49   Log-Likelihood:                 22.556
No. Observations:                 569   AIC:                            -17.11
Df Residuals:                     555   BIC:                             43.70
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
concave points_worst      